In [2]:
from Bio import SeqIO
from Bio import Seq
import os

# Input files
# input_fasta = "/scratch/bmarine2/canFam4.fa"
input_fasta = "/data/CEM/smacklab/genomes/bismark/UU_Cfam_GSD_1.0_ROSY/UU_Cfam_GSD_1.0_ROSY.fa"
output_fasta = "canFam4_modified.fa"
snp_dir = "/scratch/bmarine2/meQTL-241101-PQLSeq/metaData_SNPs/"
edited_chrs_file = "chrs_edited.tsv"
edited_snps_file = "edited_snps.txt"  # File to save edited SNPs

# Track edited chromosomes
edited_chrs = set()
if os.path.exists(edited_chrs_file):
    with open(edited_chrs_file, "r") as f:
        edited_chrs.update(line.strip() for line in f)

# Edited SNPs tracker
edited_snps = []


In [3]:
def process_chromosome(chr_name, sequence):
    snp_file = f"{snp_dir}/{chr_name}_SNPs.tsv"
    if not os.path.exists(snp_file):
        print(f"No SNP data for {chr_name}")
        return sequence, False, []

    print(f"Processing {chr_name}...")

    # Read SNP data
    with open(snp_file, "r") as f:
        snp_data = [line.strip().split() for line in f.readlines()[1:]]  # Space-delimited

    # Convert sequence to a mutable list
    seq_list = list(sequence)
    snps_edited = []  # Track SNPs edited in this chromosome

    # Process T->C and A->G SNPs with conditions
    for row in snp_data:
        try:
            _, _, pos, ref, alt, _ = row  # Unpack SNP data
            pos = int(pos) - 1  # Adjust for 0-based indexing
            ref = ref.strip("\"").upper()  # Clean and uppercase REF
            alt = alt.strip("\"").upper()  # Clean and uppercase ALT

            # Edit T->C if downstream is G
            if ref == "T" and alt == "C" and seq_list[pos + 1].upper() == "G":
                if seq_list[pos].upper() == "T":
                    seq_list[pos] = "C"
                    snps_edited.append(f"{chr_name}_{pos + 1}")  # Log SNP as 1-based

            # Edit A->G if upstream is C
            if ref == "A" and alt == "G" and seq_list[pos - 1].upper() == "C":
                if seq_list[pos].upper() == "A":
                    seq_list[pos] = "G"
                    snps_edited.append(f"{chr_name}_{pos + 1}")  # Log SNP as 1-based

        except IndexError:
            # Ignore IndexErrors (e.g., at the start or end of the sequence)
            continue

    # Convert list back to a string
    return "".join(seq_list), bool(snps_edited), snps_edited



In [4]:
# Main loop to process each chromosome
for record in SeqIO.parse(input_fasta, "fasta"):
    chr_name = record.id
    output_file = f"fastas_edited/{chr_name}_edited.fasta"

    if chr_name in edited_chrs:
        print(f"{chr_name} already processed. Skipping...")
        continue

    # Process the chromosome
    new_sequence, edited, snps_edited = process_chromosome(chr_name, str(record.seq))
    if edited:
        record.seq = Seq.Seq(new_sequence)
        edited_chrs.add(chr_name)
        edited_snps.extend(snps_edited)  # Add edited SNPs for this chromosome

    # Write the processed chromosome to a separate file
    with open(output_file, "w") as output_handle:
        SeqIO.write(record, output_handle, "fasta")

    print(f"Chromosome {chr_name} saved to {output_file}")

# Save the edited chromosomes list
with open(edited_chrs_file, "w") as f:
    for chr_name in edited_chrs:
        f.write(f"{chr_name}\n")

# Save the list of edited SNPs
with open(edited_snps_file, "w") as f:
    for snp in edited_snps:
        f.write(f"{snp}\n")

print(f"FASTA editing complete. Each chromosome saved separately.")
print(f"List of edited SNPs saved to {edited_snps_file}.")

Processing chr1...
Chromosome chr1 saved to fastas_edited/chr1_edited.fasta
Processing chr2...
Chromosome chr2 saved to fastas_edited/chr2_edited.fasta
Processing chr3...
Chromosome chr3 saved to fastas_edited/chr3_edited.fasta
Processing chr4...
Chromosome chr4 saved to fastas_edited/chr4_edited.fasta
Processing chr5...
Chromosome chr5 saved to fastas_edited/chr5_edited.fasta
Processing chr6...
Chromosome chr6 saved to fastas_edited/chr6_edited.fasta
Processing chr7...
Chromosome chr7 saved to fastas_edited/chr7_edited.fasta
Processing chr8...
Chromosome chr8 saved to fastas_edited/chr8_edited.fasta
Processing chr9...
Chromosome chr9 saved to fastas_edited/chr9_edited.fasta
Processing chr10...
Chromosome chr10 saved to fastas_edited/chr10_edited.fasta
Processing chr11...
Chromosome chr11 saved to fastas_edited/chr11_edited.fasta
Processing chr12...
Chromosome chr12 saved to fastas_edited/chr12_edited.fasta
Processing chr13...
Chromosome chr13 saved to fastas_edited/chr13_edited.fasta
